In [3]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from natsort import natsorted
import os
import time
import sys
import copy
sys.path.append('../')
sys.path.insert(0, '../optical_tracking')
sys.path.insert(0, '../util')
from phantom_registration import manual_registration, draw_registration_result
from Solver import solver
sol = solver()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [56]:
#test
depth_threshold = 0.3# meters
disp = np.load('/home/shc/Desktop/data/0308/opti_3/disp/150.npy')
Q = np.load('../params/Q.npy')
disp = cv2.flip(disp, 1)

point_cloud = cv2.reprojectImageTo3D(disp, Q)
point_cloud = point_cloud.reshape(-1, 3)

# Filter out the points beyond the depth threshold
filtered_point_cloud = point_cloud[abs(point_cloud[:, 2]) < abs(depth_threshold)]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(filtered_point_cloud)
o3d.visualization.draw_geometries([pcd])

FileNotFoundError: [Errno 2] No such file or directory: '/home/shc/Desktop/data/0308/opti_2/disp/150.npy'

In [124]:
# pcd = o3d.io.read_point_cloud('../data/phantom_point-cloud_data/phacon_exp_3.ply')
pcd_ = o3d.io.read_point_cloud('/home/shc/Desktop/data/0308/opti_3/zed_pointcloud/30.ply')
pcd_c = o3d.io.read_point_cloud('/home/shc/Desktop/data/0308/opti_3/cres_pointcloud/30.ply')

o3d.visualization.draw_geometries([pcd_, pcd_c])

Path of data

In [4]:
# disparity_dir = '../../optimization_data/disp'
# save_dir = '../../optimization_data/pointcloud'

opti_dir = '/home/shc/Desktop/data/0308/opti_3/'
disparity_dir = os.path.join(opti_dir, 'disp')
save_dir = os.path.join(opti_dir, '/cres_pointcloud')
Q = np.load('../params/Q.npy')

**Generate cres pointcloud from the disparity maps and filter the pointcloud by depth.**

In [ ]:

count = 0
start = time.time()
# Define the depth threshold
depth_threshold = 300 /1000 # meters

# Sequence of images from the left/right dir
disparities = natsorted([os.path.join(disparity_dir, f) for f in os.listdir(disparity_dir) if ".npy" in f])

for disp_map in disparities:
    disp = np.load(disp_map)
    disp = cv2.flip(disp, 1)
    point_cloud = cv2.reprojectImageTo3D(disp, Q)
    point_cloud = point_cloud.reshape(-1, 3)

    # Filter out the points beyond the depth threshold
    filtered_point_cloud = point_cloud[abs(point_cloud[:, 2]) < abs(depth_threshold)]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(filtered_point_cloud)
    
    # # Radius outlier removal:
    # pcd_rad, ind_rad = pcd.remove_radius_outlier(nb_points=16, radius=0.05)
    # outlier_rad_pcd = pcd.select_by_index(ind_rad, invert=True)
    # outlier_rad_pcd.paint_uniform_color([1., 0., 1.])

    # Statistical outlier removal:
    pcd_stat, ind_stat = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                 std_ratio=2.0)
    
    outlier_stat_pcd = pcd.select_by_index(ind_stat, invert=True)
    outlier_stat_pcd.paint_uniform_color([0., 0., 1.])

    # o3d.visualization.draw_geometries([pcd_stat])

    save_path = os.path.join(save_dir, f'{count}.ply')
    o3d.io.write_point_cloud(save_path, pcd_stat)
    print('Finished saving: '+f'{count}.ply')
    
    count += 1
end = time.time()
print(f'Done with pointcloud saving in {end - start}s')

Filter ZED pointcloud

In [17]:
pointcloud_dir = '/home/shc/Desktop/data/0308/opti_3/zed_pointcloud/'
save_dir = pointcloud_dir

depth_threshold = 0.3
count = 0

pointclouds = natsorted([os.path.join(pointcloud_dir, f) for f in os.listdir(pointcloud_dir) if ".ply" in f])
for pointcloud in pointclouds:

    pcd_zed = o3d.io.read_point_cloud(os.path.join(pointcloud_dir, pointcloud))

    # Statistical outlier removal:  
    pcd_zed, ind_stat = pcd_zed.remove_statistical_outlier(nb_neighbors=20,
                                                std_ratio=2.0)
    
    np_pcd_zed = np.asarray(pcd_zed.points)

    distances = np.sqrt(np.sum(np_pcd_zed**2, axis=1))
    filtered_point_cloud = np_pcd_zed[distances <= depth_threshold]
    # filtered_point_cloud = np_pcd_zed[abs(np_pcd_zed[:, 2]) < abs(depth_threshold)]

    pcd_zed.points = o3d.utility.Vector3dVector(filtered_point_cloud)
    save_path = os.path.join(save_dir, pointcloud)
    # o3d.io.write_point_cloud(save_path, pcd_zed)
    print('Finished saving: '+f'{count}.ply')
    count += 1
    if count > 128:
        o3d.io.write_point_cloud(save_path, pcd_zed)
        o3d.visualization.draw_geometries([pcd_zed])

Finished saving: 0.ply
Finished saving: 1.ply
Finished saving: 2.ply
Finished saving: 3.ply
Finished saving: 4.ply
Finished saving: 5.ply
Finished saving: 6.ply
Finished saving: 7.ply
Finished saving: 8.ply
Finished saving: 9.ply
Finished saving: 10.ply
Finished saving: 11.ply
Finished saving: 12.ply
Finished saving: 13.ply
Finished saving: 14.ply
Finished saving: 15.ply
Finished saving: 16.ply
Finished saving: 17.ply
Finished saving: 18.ply
Finished saving: 19.ply
Finished saving: 20.ply
Finished saving: 21.ply
Finished saving: 22.ply
Finished saving: 23.ply
Finished saving: 24.ply
Finished saving: 25.ply
Finished saving: 26.ply
Finished saving: 27.ply
Finished saving: 28.ply
Finished saving: 29.ply
Finished saving: 30.ply
Finished saving: 31.ply
Finished saving: 32.ply
Finished saving: 33.ply
Finished saving: 34.ply
Finished saving: 35.ply
Finished saving: 36.ply
Finished saving: 37.ply
Finished saving: 38.ply
Finished saving: 39.ply
Finished saving: 40.ply
Finished saving: 41.ply
Fi

KeyboardInterrupt: 

**Do phantom registration for all pointcloud**

In [7]:
def phantom_registration(pointcloud_dir, T_c_p_path):

    pcd_source_init = o3d.io.read_point_cloud(os.path.join(pointcloud_dir, "0.ply"))
    pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")
    pcd_target = pcd_target.voxel_down_sample(voxel_size=0.001)
    cam2ph_list = []
    count = 0

    # manual registration to initialize
    init_result = manual_registration(pcd_source_init, pcd_target)
    # draw_registration_result(pcd_source_init, pcd_target, result.transformation)

    pointclouds = natsorted([os.path.join(pointcloud_dir, f) for f in os.listdir(pointcloud_dir) if ".ply" in f])
    # pointclouds = pointclouds[15:]
    print("Apply point-to-plane ICP")

    loss = o3d.pipelines.registration.TukeyLoss(k=0.1)

    for pointcloud in pointclouds:
        pcd_source = o3d.io.read_point_cloud(pointcloud)
        
        threshold = 1.0

        trans_init_icp = init_result.transformation

        result = o3d.pipelines.registration.registration_icp(
            pcd_source, pcd_target, threshold, trans_init_icp,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(loss))

        print(f"Transformation is ({count}):")
        print(result.transformation)
        print(count)
        draw_registration_result(pcd_source, pcd_target, result.transformation)
        ph2cam = result.transformation
        cam2ph = sol.invTransformation(ph2cam)
        cam2ph[:3, 3] = cam2ph[:3, 3]*1000
        cam2ph_list.append(cam2ph)
        
        init_result = result
        count += 1

        # # print(T_o_p.shape)
        # if count == 30:
        #     break
    T_c_p = np.array(cam2ph_list)
    np.save(f'../data/post_optimization_data/{T_c_p_path}', T_c_p)


In [8]:
# pointcloud_dir = '../../optimization_data/pointcloud/'
pointcloud_dir = '/home/shc/Desktop/data/0308/opti_3/zed_pointcloud/'
phantom_registration(pointcloud_dir, 'T_c_p_opti3zed.npy')

Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #15488 (-0.012, -0.044, -0.27) to add in queue.
[Open3D INFO] Picked point #9421 (0.011, -0.034, -0.28) to add in queue.
[Open3D INFO] Picked point #19890 (0.019, -0.052, -0.27) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #31921 (-0.02, 0.0023, 0.0059) to add in queue.
[Open3D INFO] Picked point #2287 (0.0057, 0.012, 0.0017) to add in queue.
[Open3D INFO] Picked point #26320 (0.0051, 0.0074, 0.024) to add in queue.

Compute a rough transform using the correspondences given by user
Perform point-to-point ICP refinement

Apply point-to-plane ICP


In [107]:
# T_c_p_opti3zed = np.load('../data/post_optimization_data/T_c_p_opti3.npy')
# pcd_source = o3d.io.read_point_cloud('/home/shc/Desktop/data/0308/opti_3/zed_pointcloud/35.ply')
# pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")
# pcd_target = pcd_target.voxel_down_sample(voxel_size=0.001)
# threshold = 0.001
# result = o3d.pipelines.registration.registration_icp(
#             pcd_source, pcd_target, threshold, T_c_p_opti3zed[34],
#             o3d.pipelines.registration.TransformationEstimationPointToPlane())

# draw_registration_result(pcd_source, pcd_target, result.transformation)

In [39]:
t = np.load('../data/post_optimization_data/T_c_p_opti3zed.npy')
np.linalg.norm(t[40, :3, 3])

pcd_source_init = o3d.io.read_point_cloud(os.path.join(pointcloud_dir, "40.ply"))
pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")
pcd_target = pcd_target.voxel_down_sample(voxel_size=0.001)

# manual registration to initialize
# result = manual_registration(pcd_source_init, pcd_target)
k = result.transformation
print(t[40],'\n', np.linalg.norm(k[:3, 3])*1000)

[[ 2.35429261e-01 -8.99441547e-01  3.68209134e-01  2.96204016e+02]
 [-8.68335735e-01 -3.64833482e-01 -3.35990448e-01  3.98101198e+00]
 [ 4.36538789e-01 -2.40627166e-01 -8.66909714e-01 -6.01037772e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 
 233.58219789334518


In [9]:
def getRealPose(idx, pd_data):
    df = pd_data
    pose_x = df['pose.position.x'][idx]*1000
    pose_y = df['pose.position.y'][idx]*1000
    pose_z = df['pose.position.z'][idx]*1000
    orin_x = df['pose.orientation.x'][idx]
    orin_y = df['pose.orientation.y'][idx]
    orin_z = df['pose.orientation.z'][idx]
    orin_w = df['pose.orientation.w'][idx]
    pose = np.array([pose_x, pose_y, pose_z, orin_x, orin_y, orin_z, orin_w])
    # print(real_pose, '\n', pose)
    return pose

In [10]:
import pandas as pd
from scipy.spatial.transform import Rotation as R

params_save_dir = '../data/post_optimization_data/'
T_p_pb = np.load('../params/phacon2pan_0227.npy')
T_pb_p = sol.invTransformation(T_p_pb)

pose_pan_data = os.path.join(opti_dir, 'fwd_pose_pan.csv')
pose_cam_data = os.path.join(opti_dir, 'fwd_pose_camhand.csv')

df_pan = pd.read_csv(pose_pan_data)
df_cam_hand = pd.read_csv(pose_cam_data)

assert len(df_pan == df_cam_hand)
num_frames = len(df_pan)

T_o_p_batch = np.zeros([num_frames, 4, 4])
T_o_cb_batch = np.zeros([num_frames, 4, 4])
for i in range(num_frames):
    quaternion_pb = getRealPose(i, df_pan)
    quaternion_cb = getRealPose(i, df_cam_hand)
    _, T_o_pb = sol.seven2trans(quaternion_pb)
    _, T_o_cb = sol.seven2trans(quaternion_cb)
    T_o_p_batch[i, :, :] = T_o_pb @ T_pb_p
    T_o_cb_batch[i,:,:] = T_o_cb

np.save(os.path.join(params_save_dir, 'T_o_p_opti3.npy'), T_o_p_batch)
np.save(os.path.join(params_save_dir, 'T_o_cb_opti3.npy'), T_o_cb_batch)
